<a href="https://colab.research.google.com/github/ronaldehido/BirdNET-colab/blob/main/RON_BirdNET_1_Inferencia(batch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ejecución Modelo Personalizado CNN en BirdNET Analyzer**
versión 2.1 (20-02-2025)
update (14/03/2025)

### *Ron A. Fernández-Gómez - EPM SECIHTI*
(puntuacion omitida)
Este es un flujo de trabajo en Google Colab escrito en Python para ejecutar modelos personalizados basados en Redes Neuronales Convolucionales (CNNs) para detección y clasificación de vocalizaciones de fauna utilizando la herramienta BirdNET Analyzer para uso con Interfaz de línea de comando (CLI).
Para mayores detalles sobre BirdNET Analyzer y usos del proyecto BirdNET consultar: https://github.com/kahst/BirdNET-Analyzer


Aunque los modelos compatibles con BirdNET Analyzer se generan en Tensorflow, algunos están en formato .tflite que pueden tener algunas limitaciones para optimizar procesos de inferencias basados en el uso de paralelización en GPU, delegando los procesos a la CPU. Por lo tanto en este flujo buscaremos trabajar en un entorno de ejecución de colab con recursos disponibles en CPU. Se recomienda usar v2-8 TPU que cuenta con la virtualización de 96 procesadores CPU.

Nota(Update: 14/03/2025): dado a las actualizaciones del respositorio de BirdNET que cambiaron los nombres de directorios y funciones, se usa en este ejemplo una copia del respositorio previo de BirdNET alojado en mi repositorio personal de la versión

# 1. INFERENCIA: Deteccion de vocalizaciones por lotes
Esta etapa corresponde a la ejecución de un modelo CNN por lotes. Permite ejecutar el modelo en una carpeta que contenga varios archivos de audio.

## Paqueterías y dependencias necesarias
Estas dependencias facilitaran el funcionamiento del analizador

In [ ]:
!pip install ffmpeg
!pip install librosa
!pip install resampy
!pip install tensorflow==2.15.0
!pip install keras-tuner

##*Conectar a Google Drive y definir rutas*
Primero vincularemos Google Drive con lo que definirimos las rutas de acceso a los archivos de trabajo y de salida de resultados durante la sesión de trabajo. En el Drive tedremos los archivos a analizar y los modelos personalizados a ejecutar.

In [ ]:
# Conectar con google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Para trabajar con varios archivos hay que definir
#Rutas de entrada y salida
audio_path = '/ruta/a/la/carpeta/de/archivos' #definir esta ruta '/ruta/a/la/carpeta/de/archivos'
output_folder = '/ruta/a/la/carpeta/donde/guardar/los/resultados' #definir esta ruta '/ruta/a/la/carpeta/donde/guardar/los/resultados'

#Ruta al modelo (en caso de usar un modelo personalizado)
my_model = '/ruta/al/modelo/personalizado.tflite' #definir esta ruta '/ruta/al/modelo/personalizado.tflite'

## Clonar BirdNET Analyzer
Clonamos el respositorio de BirdNET Analyzer en nuestro espacio de trabajo a partir de GitHub
Nota (14/03/2025): Dado que el repsitorio de BirdNET fue actualizado, el codigo original no es funcional por el momento, por lo que usaremos una copia previa de BirdNET alojada en mi repositorio.

In [ ]:
!git clone https://github.com/ronaldehido/cloned-BirdNET-01-02-2025 #copia clonada de version previa de BirdNET en mi repositorio personal
#por defecto se clonara dentro de nuestro directorio de trabajo /content

In [ ]:
#se establece el nuevo directorio de trabajo donde estan las funciones en el folder /content/cloned-BirdNET-Analyzer
%cd cloned-BirdNET-01-02-2025


In [ ]:
#revisamos el directorio y confirmamos
!pwd

## Ejecutar inferencia con BirdNET Analyzer
Clonamos el respositorio de BirdNET Analyzer en nuestro espacio de trabajo a partir de GitHub
Nota (14/03/2025): Dado que el repsitorio de BirdNET fue actualizado, el codigo original no es funcional por el momento, por lo que usaremos una copia previa de BirdNET alojada en mi repositorio.

### Ejemplo de inferencia con un archivo
Usamos un ejemplo incluido en BirdNET para hacer una prueba rapida de que la funcion corre correctamente. La prueba se usa corriendo el modelo base (GLOBAL V2.4, June 2023). El resultado se obtendra como un arhcivo de texto en la carpeta de trabajo. Lo revisamos y verificamos que todo sea correcto.

In [ ]:
#Hacemos una prueba rapida con un ejemplo incluido (archivo example) para verificar el funcionamiento correcto de la funcion "analyze"
!python -m birdnet_analyzer.analyze \
        --combine_results

### Inferencia en lote con modelo personalizado
Ejecutamos BirdNET sobre un directorio completo y ajustamos parametros adicionales segun nuestra necesidad, ademas incluimos un modelo entrenado personalizado.

In [ ]:
# Procesar una carpeta con archivos y el modelo personalizado, en este caso cargamos el modelo de WildMon dando la ruta del archivo
!python -m birdnet_analyzer.analyze \
        --i "{audio_path}" \
        --o "{output_folder}" \
        --classifier "{my_model}" \
        --min_conf 0.5 \
        --threads 96 \
        --batchsize 32 \
        --combine_results

        #Parametros
        # --i "{audio_path}" \ #ruta asignada de archivos de ingreso (input)
        # --o "{output_folder}" \ #ruta asignada de archivos de salida (output)
        # --classifier "{my_model}" \ #ruta asignada donde se encuentra el modelo personalizado, si no se declara se usara por defecto el modelo base (GLOBAL).
        # --min_conf 0.5 \ #definimos el minimo de confianza
        # --threads 96 \ #denifimos la cantidad de hilos de procesadores CPU a utilizar, por ejemplo 96 el maximo del entorno de ejecución
        # --batchsize 32 \ #definimos la cantidad de archivos a anlizar al tiempo.
        # --combine_results #solo se declara si se quiere unificar los resultados en un unico archivo csv

In [ ]:
print("\n🎯 Proceso finalizado correctamente. ¡Buen trabajo! 🚀")

Fin.